In [52]:
import folium
import gpxpy

gpx_file = open("gpx_data/20210407-115732.gpx","r")
data = gpxpy.parse(gpx_file)
pts = data.tracks[0].segments[0].points

start = [pts[0].latitude, pts[0].longitude]
stop = [pts[-1].latitude, pts[-1].longitude]

m = folium.Map(location = [pts[0].latitude, pts[0].longitude], zoom_start = 14, tiles = "CartoDB dark_matter")

# Imrpime a rota, criando um gráfico de cores baseado na velocidade
# Variando do verde ao vermelho, conforme aumenta a velocidade
for index in range(len(pts)-1):
    max_speed = max([pts[i].speed for i in range(len(pts))]) * 3.6
    factor = int((255 // max_speed)) - 1
    speed = int((pts[index].speed) * 3.6 ) * factor
    color = '#{:02x}{:02x}{:02x}'.format(0 + speed, 255 - speed, 0)
    coord = [(pts[index].latitude,pts[index].longitude),(pts[index + 1].latitude,pts[index + 1].longitude)]
    folium.PolyLine(coord, color=color, weight=7).add_to(m)

count = False
time = 0
stop_points = []
time_list = []

# Analisa todos os pontos de parada, e o tempo de duração em cada ponto, criando uma lista com as coordenadas
for index in range(len(pts)):
    
    if pts[index].speed == 0:
        if not count:
            count = True
            time += 1
            stop_points.append((pts[index].latitude, pts[index].longitude))
        else:
            time +=1
    else:
        if count:
            count = False
            time_list.append(time)
            time = 0

    if index + 1 == len(pts) and count:
        time_list.append(time)

# Imprime no mapa os pontos de paradas com circulos vermelhos
for index in range(len(stop_points)):
    lat, lng = stop_points[index]
    msg = '{} segundos'.format(time_list[index])
    folium.CircleMarker(location=[lat, lng], color="red", radius=15, fill=True, popup=msg).add_to(m)

# Imprime os pontos de saida e chegada
folium.Marker(start, icon=folium.Icon(color="green")).add_to(m)
folium.Marker(stop, icon=folium.Icon(color="red")).add_to(m)
     
image = folium.Figure(width = 1280, height = 720)
image.add_child(m)
image
